# Dynamic Programming and Optimal Control
## Problem Set 3, Problem 4

ETH Zurich \
Institute for Dynamic Systems and Control 

Revision history \
[08.10.2013, Dario Brescianini]         First version (based on old programming exercises) \
[10.09.2022, MR]   Convert to Python

**Python script that solves Problem 4 of Problem Set 3 by applying applying the Label Correcting method and A* algorithm.** 

**We used [NumPy](https://numpy.org/) and [SciPy](https://scipy.org/) packages, please install in your Python environment. You can install these packages using `pip install` or `conda install` command depending on your package manager. You can also find the installation guide in the package websites or documentations.**

In [2]:
import scipy.io
import numpy as np
import time

### Label correcting algorithm function

In [3]:
def lca(A,startNode,terminalNode):
    # Executes Label Correcting algorithm (Book Dynamic Programming and Optimal
    # Control, Bertsekes, page 81) using the depth-first method.

    # Input:
    #   A               [NxN] matrix, where the element A(i,j) = a_ij is the cost
    #                   to move from node i to j.
    #   startNode       Start node of desired shortest path, scalar from 1 to N.
    #   terminalNode    Terminal node of desired shortest path, scalar from 1
    #                   to N.

    # Output:
    #   optCost         Cost of the shortest path(s), scalar:
    #   optPath         Row vector containing the shortest path, e.g. 
    #                   optPath = [0 32 44 43 78 99].

    N = len(A)  # Dimension of the problem: N = total number of nodes
    d = np.ones(N)*np.inf # Vector holding label d for each node. d(i) represents
                        # the shortest path found so far from start node to i.
    d[startNode] = 0
    parent = np.ones(N)*np.inf  # Vector containing the parent of the shortest path
                              # found so far for each node.
    parent[startNode] = -1
    OPEN = np.zeros(N)  # List cotaining all the nodes that are currently 
                      # active in the sense that they are candidates for 
                      # further examination (candidates list).
    pointerOPEN = 1     # Pointer which always points to the last element in OPEN.
    OPEN[pointerOPEN] = startNode
    UPPER = np.inf      # Label dt, representing the shortest path to the end found so far.

    ## Check start and terminal node
    # Make sure that the start and terminal node are valid.
    if startNode == terminalNode:
        optCost = 0
        optPath = [startNode, terminalNode]
        return optCost, optPath # Done.

    if (startNode >= N or terminalNode >= N) or (startNode < 0 or terminalNode < 0):
        optCost = np.inf
        optPath = None
        return optCost, optPath # Done.

    # Execute algorithm
    while 1:
        # STEP 1: Remove a node i from OPEN and for each child j of i, execute STEP 2.
        i = int(OPEN[pointerOPEN])
        OPEN[pointerOPEN] = 0
        pointerOPEN = pointerOPEN - 1

        children = np.where(A[i,:] != np.inf) 
        children = children[0]
        if i in children:
            children = np.delete(children, np.where(children == i))

        for j in children:
            # STEP 2: If d_i + a_ij < min(d_j,UPPER), set d_j = d_i + a_ij and
            # set i to be the parent of j.
            if (d[i] + A[i,j] < min([d[j],UPPER])):
                d[j] = d[i] + A[i,j]

                parent[j] = int(i)

                # In addition, if j != t, place j in OPEN if it is not already
                # in OPEN, while if j == t, set UPPER to the new value d_i +
                # a_it of d_t
                if (j != terminalNode):
                    if not(j in OPEN):
                        pointerOPEN += 1
                        OPEN[pointerOPEN] = j
                else:
                    UPPER = d[j]
    
        # STEP 3: If OPEN is empty, terminate; else go to STEP 1.
        if (not pointerOPEN):
            break
    
    # UPPER is equal to the cost of the shortest path.
    optCost = UPPER

    # Construct shortest path
    # Start at terminal node and, for each node, take its parent node until we
    # find ourselves at the start node.
    optPath = [terminalNode]
    while (optPath[-1] != startNode):
        optPath.append(int(parent[int(optPath[-1])]))
    optPath.reverse()  # Reverse path: startNode -> terminalNode
    print(f"optCost, optPath: {optCost, optPath}")
    return optCost, optPath

### A* algorithm function

In [3]:
def astar(A,startNode,terminalNode):
    # [optCost, optPath] = lca(A,startNode,terminalNode)

    # Executes A* algorithm (Book Dynamic Programming and Optimal
    # Control, Bertsekes, page 87) using the depth-first method.

    # Input:
    #   A               [NxN] matrix, where the element A(i,j) = a_ij is the cost
    #                   to move from node i to j.
    #   startNode       Start node of desired shortest path, scalar from 1 to N.
    #   terminalNode    Terminal node of desired shortest path, scalar from 1
    #                   to N.

    # Output:
    #   optCost         Cost of the shortest path(s), scalar:
    #   optPath         Row vector containing the shortest path, e.g. 
    #                   optPath = [0 33 45 43 79 99].

    # Initialization
    N = len(A)          # Dimension of the problem: N = total number of nodes
    d = np.ones(N)*np.inf # Vector holding label d for each node. d(i) represents
                        # the shortest path found so far from start node to i.
    d[startNode] = 0
    parent = np.ones(N)*np.inf # Vector containing the parent of the shortest path
                             # found so far for each node.
    parent[startNode] = 0
    OPEN = np.zeros(N)  # List cotaining all the nodes that are currently 
                      # active in the sense that they are candidates for 
                      # further examination (candidates list).
    pointerOPEN = 1     # Pointer which always points to the last element in OPEN.
    OPEN[pointerOPEN] = startNode
    UPPER = np.inf      # Label dt, representing the shortest path to the end found so far.

    # Check start and terminal node
    # Make sure that the start and terminal node are valid.
    if startNode == terminalNode:
        optCost = 0
        optPath = [startNode, terminalNode]
        return optCost, optPath        # Done.

    if (startNode >= N or terminalNode >= N) or (startNode < 0 or terminalNode < 0):
        optCost = np.inf
        optPath = None
        return optCost, optPath        # Done.

    # Execute algorithm
    while 1:
        # STEP 1: Remove a node i from OPEN and for each child j of i, execute STEP 2.
        i = int(OPEN[pointerOPEN])
        OPEN[pointerOPEN] = 0
        pointerOPEN = pointerOPEN - 1

        children = np.where(A[i,:] != np.inf) 
        children = children[0]
        if i in children:
            children = np.delete(children, np.where(children == i))
      
        for j in children:
            # STEP 2: If d_i + a_ij < and  d_i + a_ij + h_j < UPPER, 
            # set d_j = d_i + a_ij and set i to be the parent of j.
            if (d[i] + A[i,j] < d[j] and d[i] + A[i,j] + abs(j-terminalNode) < UPPER):
                d[j] = d[i] + A[i,j]
                parent[j] = i

                # In addition, if j ~= t, place j in OPEN if it is not already
                # in OPEN, while if j == t, set UPPER to the new value d_i +
                # a_it of d_t
                if (j != terminalNode):
                    if not (j in OPEN):
                        pointerOPEN += 1
                        OPEN[pointerOPEN] = j
                else:
                    UPPER = d[j]
      
        #  STEP 3: If OPEN is empty, terminate; else go to STEP 1.
        if (not pointerOPEN):
            break
  
    # DONE.
    # UPPER is equal to the cost of the shortest path.
    optCost = UPPER
    # Construct shortest path
    # Start at terminal node and, for each node, take its parent node until we
    # find ourselves at the start node.
    optPath = [terminalNode]
    while optPath[-1] != startNode:
        optPath.append(int(parent[int(optPath[-1])]))
    optPath.reverse()  # Reverse path: startNode -> terminalNode
    return optCost, optPath


### Initialize variables

In [4]:
mat = scipy.io.loadmat('A.mat')    
A = mat['A'] # Load matrix A that contains all the transition costs A(i,j) = a_ij to get from i to j.
N = len(A)  # Dimension of the problem: N = total number of nodes

### Define start and terminal node

In [5]:
# Default values:
#   startNode = 0
#   terminalNode = 99

# Minimum path length (minimum total cost): 100
# Path: 0 -> 2 -> 40 -> 50 -> 99

startNode = 0      
terminalNode = 99

### Label Correcting Algorithm
Solve shortest path problem using the Label Correcting Algorithm

In [6]:
t = time.time()
[optCost1, optPath1] = lca(A,startNode,terminalNode)   # Your implementation of the Label Correcting aglorithm.
time1 = time.time() - t

optCost, optPath: (100.0, [0, 2, 40, 50, 99])


### A* Algorithm
Solve shortest path problem using the A* Algorithm

In [7]:
t = time.time()
[optCost2, optPath2] = astar(A,startNode,terminalNode) # Your implementation of the A* algorithm.
time2 = time.time() - t

### Print results

In [8]:
print('Results')
print('Problem with ',N,' nodes.')
print('Optimal path from node ',startNode,' to ',terminalNode,':')
print('\033[1mLabel Correcting Algorithm\033[0m')
print('Execution time: ',time1,' s.')
print('Path: ', optPath1)
print('\033[1mA* Algorithm\033[0m')
print('Execution time: ',time2,'s  (',time2/time1,' times the time for method 1).')
print('Minimum path length (minimum total cost): ',optCost2)
print('Path: ', optPath2)

Results
Problem with  100  nodes.
Optimal path from node  0  to  99 :
Label Correcting Algorithm
Execution time:  0.040026187896728516  s.
Path:  [0, 2, 40, 50, 99]
A* Algorithm
Execution time:  0.006212949752807617 s  ( 0.1552221202987813  times the time for method 1).
Minimum path length (minimum total cost):  100.0
Path:  [0, 2, 40, 50, 99]
